Dataset

In [1]:
import os
from PIL import Image
import json

def load_images_from_directory(root_path: str):
    """
    Load images from a directory with subfolders named after ImageNet labels.
    Return a list of (image, label, filename) triples.
    """
    dataset = []
    
    # Iterate over each subfolder
    for label in os.listdir(root_path):
        label_path = os.path.join(root_path, label)
        
        # Check if it's indeed a folder
        if os.path.isdir(label_path):
            
            # Iterate over each image in the subfolder
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                
                # Check if it's an image file
                if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img = Image.open(image_path)
                    dataset.append((img, label, image_file))  # Add image filename here
    
    return dataset


current_dir = "/home/workstation/code/XAImethods/CAIN"
detail_dir = "/imagenet/val_images10k_attack/defocus_blur/2_vit"

dataset_path = f"{current_dir}{detail_dir}"



dataset = load_images_from_directory(dataset_path)
with open(f"{current_dir}/imagenet/imagenet_class_index.json", "r") as f:
    imagenet_class_index = json.load(f)

label_to_index_description = {v[0]: (k, v[1]) for k, v in imagenet_class_index.items()}
import warnings
warnings.filterwarnings('ignore')
from codecarbon import track_emissions
from torchvision import transforms
from datasets import load_dataset
from pytorch_grad_cam import run_dff_on_image
from pytorch_grad_cam import (
    GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
    AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
    LayerCAM, FullGrad, GradCAMElementWise
)
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import cv2
import torch
from typing import List, Callable, Optional
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA!")
else:
    device = torch.device("cpu")
    print("Using CPU!")
# dataset = load_dataset("huggingface/cats-image")
# image = dataset["test"]["image"][0]
# img_tensor = transforms.ToTensor()(image)

""" Model wrapper to return a tensor"""
class HuggingfaceToTensorModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(HuggingfaceToTensorModelWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model(x).logits

""" Translate the category name to the category index.
    Some models aren't trained on Imagenet but on even larger datasets,
    so we can't just assume that 761 will always be remote-control.

"""
def category_name_to_index(model, category_name):
    name_to_index = dict((v, k) for k, v in model.config.id2label.items())
    return name_to_index[category_name]
    
""" Helper function to run GradCAM on an image and create a visualization.
    (note to myself: this is probably useful enough to move into the package)
    If several targets are passed in targets_for_gradcam,
    e.g different categories,
    a visualization for each of them will be created.
    
"""

Using CUDA!


' Helper function to run GradCAM on an image and create a visualization.\n    (note to myself: this is probably useful enough to move into the package)\n    If several targets are passed in targets_for_gradcam,\n    e.g different categories,\n    a visualization for each of them will be created.\n    \n'

functions

In [2]:
# #    GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
#     AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
#     LayerCAM, FullGrad, GradCAMElementWise

# #    GradCAM, HiResCAM, GradCAMPlusPlus, XGradCAM, LayerCAM
CAM_ALGORITHM = GradCAM
cam_algorithm_name = CAM_ALGORITHM.__name__

# @track_emissions()
def run_grad_cam_on_image(model: torch.nn.Module,
                          target_layer: torch.nn.Module,
                          targets_for_gradcam: List[Callable],
                          input_tensor: torch.nn.Module,
                          input_image: Image,
                          reshape_transform: Optional[Callable] = None,
                          method: Callable = CAM_ALGORITHM):
    with method(model=HuggingfaceToTensorModelWrapper(model),
                target_layers=[target_layer],
                reshape_transform=reshape_transform) as cam:

        # Replicate the tensor for each of the categories we want to create Grad-CAM for:
        repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

        batch_results = cam(input_tensor=repeated_tensor,
                            targets=targets_for_gradcam)
        results = []
        grayscale_cams = []
        for grayscale_cam in batch_results:
            visualization = show_cam_on_image(np.float32(input_image) / 255,
                                              grayscale_cam,
                                              use_rgb=True)
            # Make it weight less in the notebook:
            visualization = cv2.resize(visualization,
                                       (visualization.shape[1] // 2, visualization.shape[0] // 2))
            results.append(visualization)
            grayscale_cams.append(grayscale_cam)
        return np.hstack(results), grayscale_cams
    
def print_top_categories(model, img_tensor, top_k=5):
    logits = model(img_tensor.unsqueeze(0)).logits
    indices = logits.cpu()[0, :].detach().numpy().argsort()[-top_k :][::-1]
    for i in indices:
        print(f"Predicted class {i}: {model.config.id2label[i]}")

# Generate targets_for_gradcam based on model's predictions
def get_top_k_targets(model, input_tensor, k=5):
    logits = model(input_tensor.unsqueeze(0)).logits
    top_k_indices = logits[0].argsort(descending=True)[:k].cpu().numpy()
    return [ClassifierOutputTarget(index) for index in top_k_indices]

import os
from tqdm import tqdm
from collections import defaultdict
import gc
###################
from transformers import ViTFeatureExtractor, ViTForImageClassification
from functools import partial
import torch
from PIL import Image
import datetime

###################

def reshape_transform_vit_huggingface(x):
    activations = x[:, 1:, :]
    activations = activations.view(activations.shape[0],
                                   12, 12, activations.shape[2])
    activations = activations.transpose(2, 3).transpose(1, 2)
    return activations



model_name = "google/vit-large-patch32-384"

# Initialize tracking variables
start_time = datetime.datetime.now()
emissions_reports = []




def ensure_rgb(img):
    if img.mode != 'RGB':
        return img.convert('RGB')
    return img

def is_valid_image_file(filepath):
    """Check if the file is a valid image file."""
    try:
        with Image.open(filepath) as img:
            img.verify()  # verify that it is a valid image
        return True
    except:
        return False

BATCH_SIZE = 100
num_batches = len(dataset) // BATCH_SIZE + (1 if len(dataset) % BATCH_SIZE != 0 else 0)

save_dir = f"{current_dir}/results/{detail_dir}/{model_name}/{cam_algorithm_name}"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, len(dataset))

    model = ViTForImageClassification.from_pretrained('google/vit-large-patch32-384').to(device)

    target_layer_dff = model.vit.layernorm
    target_layer_gradcam = model.vit.encoder.layer[-2].output




    for idx in range(start_idx, end_idx):
        img, label, filename = dataset[idx]
        try:
            torch.cuda.empty_cache()
            img = ensure_rgb(img)
            image_resized = img.resize((384, 384))
            tensor_resized = transforms.ToTensor()(image_resized).to(device)
            # resize_transform = transforms.Resize((480, 640))
            # img = resize_transform(img)
            # img_tensor = transform(img).to(device)
            #print(img_tensor.shape)


            index_description = label_to_index_description.get(label)
            if index_description is None:
                print(f"Warning: Label '{label}' not found in the JSON file!")
                continue

            index_str, description = index_description
            index = int(index_str)
            dynamic_targets_for_gradcam = [ClassifierOutputTarget(index)]

            
            gradcam_result, grayscale_cams = run_grad_cam_on_image(
                model=model,
                target_layer=target_layer_gradcam,
                targets_for_gradcam=dynamic_targets_for_gradcam,
                input_tensor=tensor_resized,
                input_image=image_resized,
                reshape_transform=reshape_transform_vit_huggingface
            )

            logits = model(tensor_resized.unsqueeze(0)).logits
            top_indices = logits[0].argsort(descending=True)[:5].cpu().numpy()
            predictions = {index: {"score": logits[0][index].item(), "label": model.config.id2label[index]} for index in top_indices}
            
            img_dir = os.path.join(save_dir, filename.rsplit('.', 1)[0])
            if not os.path.exists(img_dir):
                os.makedirs(img_dir)

            img_name = os.path.join(img_dir, "original.jpg")
            gradcam_name = os.path.join(img_dir, "gradcam.jpg")
            grayscale_name = os.path.join(img_dir, "grayscale.jpg")
            grayscale_npy_name = os.path.join(img_dir, "grayscale.npy")
            scores_name = os.path.join(img_dir, "scores.npy")
            info_name = os.path.join(img_dir, "info.txt")

            img.save(img_name)
            Image.fromarray(gradcam_result).save(gradcam_name)
            Image.fromarray((grayscale_cams[0] * 255).astype(np.uint8)).save(grayscale_name)
            np.save(grayscale_npy_name, grayscale_cams[0])

            scores = [data["score"] for _, data in predictions.items()]
            np.save(scores_name, scores)

            with open(info_name, 'w') as f:
                for index, data in predictions.items():
                    label = data["label"]
                    score = data["score"]
                    f.write(f"Class {index} ({label}): {score:.2f}\n")

        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print(f"CUDA OutOfMemoryError encountered for file: {filename}")
            else:
                raise e

    # del model
    torch.cuda.empty_cache()
    gc.collect()


end_time = datetime.datetime.now()


print(f"Total time: {end_time - start_time}")




100%|██████████| 62/62 [11:43<00:00, 11.35s/it]

Total time: 0:11:43.828623


In [3]:
# #    GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
#     AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
#     LayerCAM, FullGrad, GradCAMElementWise

# #    GradCAM, HiResCAM, GradCAMPlusPlus, XGradCAM, LayerCAM
CAM_ALGORITHM = HiResCAM
cam_algorithm_name = CAM_ALGORITHM.__name__

# @track_emissions()
def run_grad_cam_on_image(model: torch.nn.Module,
                          target_layer: torch.nn.Module,
                          targets_for_gradcam: List[Callable],
                          input_tensor: torch.nn.Module,
                          input_image: Image,
                          reshape_transform: Optional[Callable] = None,
                          method: Callable = CAM_ALGORITHM):
    with method(model=HuggingfaceToTensorModelWrapper(model),
                target_layers=[target_layer],
                reshape_transform=reshape_transform) as cam:

        # Replicate the tensor for each of the categories we want to create Grad-CAM for:
        repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

        batch_results = cam(input_tensor=repeated_tensor,
                            targets=targets_for_gradcam)
        results = []
        grayscale_cams = []
        for grayscale_cam in batch_results:
            visualization = show_cam_on_image(np.float32(input_image) / 255,
                                              grayscale_cam,
                                              use_rgb=True)
            # Make it weight less in the notebook:
            visualization = cv2.resize(visualization,
                                       (visualization.shape[1] // 2, visualization.shape[0] // 2))
            results.append(visualization)
            grayscale_cams.append(grayscale_cam)
        return np.hstack(results), grayscale_cams
    
def print_top_categories(model, img_tensor, top_k=5):
    logits = model(img_tensor.unsqueeze(0)).logits
    indices = logits.cpu()[0, :].detach().numpy().argsort()[-top_k :][::-1]
    for i in indices:
        print(f"Predicted class {i}: {model.config.id2label[i]}")

# Generate targets_for_gradcam based on model's predictions
def get_top_k_targets(model, input_tensor, k=5):
    logits = model(input_tensor.unsqueeze(0)).logits
    top_k_indices = logits[0].argsort(descending=True)[:k].cpu().numpy()
    return [ClassifierOutputTarget(index) for index in top_k_indices]

import os
from tqdm import tqdm
from collections import defaultdict
import gc
###################
from transformers import ViTFeatureExtractor, ViTForImageClassification
from functools import partial
import torch
from PIL import Image
import datetime

###################

def reshape_transform_vit_huggingface(x):
    activations = x[:, 1:, :]
    activations = activations.view(activations.shape[0],
                                   12, 12, activations.shape[2])
    activations = activations.transpose(2, 3).transpose(1, 2)
    return activations



model_name = "google/vit-large-patch32-384"

# Initialize tracking variables
start_time = datetime.datetime.now()
emissions_reports = []




def ensure_rgb(img):
    if img.mode != 'RGB':
        return img.convert('RGB')
    return img

def is_valid_image_file(filepath):
    """Check if the file is a valid image file."""
    try:
        with Image.open(filepath) as img:
            img.verify()  # verify that it is a valid image
        return True
    except:
        return False

BATCH_SIZE = 100
num_batches = len(dataset) // BATCH_SIZE + (1 if len(dataset) % BATCH_SIZE != 0 else 0)

save_dir = f"{current_dir}/results/{detail_dir}/{model_name}/{cam_algorithm_name}"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, len(dataset))

    model = ViTForImageClassification.from_pretrained('google/vit-large-patch32-384').to(device)

    target_layer_dff = model.vit.layernorm
    target_layer_gradcam = model.vit.encoder.layer[-2].output




    for idx in range(start_idx, end_idx):
        img, label, filename = dataset[idx]
        try:
            torch.cuda.empty_cache()
            img = ensure_rgb(img)
            image_resized = img.resize((384, 384))
            tensor_resized = transforms.ToTensor()(image_resized).to(device)
            # resize_transform = transforms.Resize((480, 640))
            # img = resize_transform(img)
            # img_tensor = transform(img).to(device)
            #print(img_tensor.shape)


            index_description = label_to_index_description.get(label)
            if index_description is None:
                print(f"Warning: Label '{label}' not found in the JSON file!")
                continue

            index_str, description = index_description
            index = int(index_str)
            dynamic_targets_for_gradcam = [ClassifierOutputTarget(index)]

            
            gradcam_result, grayscale_cams = run_grad_cam_on_image(
                model=model,
                target_layer=target_layer_gradcam,
                targets_for_gradcam=dynamic_targets_for_gradcam,
                input_tensor=tensor_resized,
                input_image=image_resized,
                reshape_transform=reshape_transform_vit_huggingface
            )

            logits = model(tensor_resized.unsqueeze(0)).logits
            top_indices = logits[0].argsort(descending=True)[:5].cpu().numpy()
            predictions = {index: {"score": logits[0][index].item(), "label": model.config.id2label[index]} for index in top_indices}
            
            img_dir = os.path.join(save_dir, filename.rsplit('.', 1)[0])
            if not os.path.exists(img_dir):
                os.makedirs(img_dir)

            img_name = os.path.join(img_dir, "original.jpg")
            gradcam_name = os.path.join(img_dir, "gradcam.jpg")
            grayscale_name = os.path.join(img_dir, "grayscale.jpg")
            grayscale_npy_name = os.path.join(img_dir, "grayscale.npy")
            scores_name = os.path.join(img_dir, "scores.npy")
            info_name = os.path.join(img_dir, "info.txt")

            img.save(img_name)
            Image.fromarray(gradcam_result).save(gradcam_name)
            Image.fromarray((grayscale_cams[0] * 255).astype(np.uint8)).save(grayscale_name)
            np.save(grayscale_npy_name, grayscale_cams[0])

            scores = [data["score"] for _, data in predictions.items()]
            np.save(scores_name, scores)

            with open(info_name, 'w') as f:
                for index, data in predictions.items():
                    label = data["label"]
                    score = data["score"]
                    f.write(f"Class {index} ({label}): {score:.2f}\n")

        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print(f"CUDA OutOfMemoryError encountered for file: {filename}")
            else:
                raise e

    # del model
    torch.cuda.empty_cache()
    gc.collect()


end_time = datetime.datetime.now()


print(f"Total time: {end_time - start_time}")




  0%|          | 0/62 [00:00<?, ?it/s]

100%|██████████| 62/62 [11:24<00:00, 11.03s/it]

Total time: 0:11:24.113024


In [4]:
# #    GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
#     AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
#     LayerCAM, FullGrad, GradCAMElementWise

# #    GradCAM, HiResCAM, GradCAMPlusPlus, XGradCAM, LayerCAM
CAM_ALGORITHM = GradCAMPlusPlus
cam_algorithm_name = CAM_ALGORITHM.__name__

# @track_emissions()
def run_grad_cam_on_image(model: torch.nn.Module,
                          target_layer: torch.nn.Module,
                          targets_for_gradcam: List[Callable],
                          input_tensor: torch.nn.Module,
                          input_image: Image,
                          reshape_transform: Optional[Callable] = None,
                          method: Callable = CAM_ALGORITHM):
    with method(model=HuggingfaceToTensorModelWrapper(model),
                target_layers=[target_layer],
                reshape_transform=reshape_transform) as cam:

        # Replicate the tensor for each of the categories we want to create Grad-CAM for:
        repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

        batch_results = cam(input_tensor=repeated_tensor,
                            targets=targets_for_gradcam)
        results = []
        grayscale_cams = []
        for grayscale_cam in batch_results:
            visualization = show_cam_on_image(np.float32(input_image) / 255,
                                              grayscale_cam,
                                              use_rgb=True)
            # Make it weight less in the notebook:
            visualization = cv2.resize(visualization,
                                       (visualization.shape[1] // 2, visualization.shape[0] // 2))
            results.append(visualization)
            grayscale_cams.append(grayscale_cam)
        return np.hstack(results), grayscale_cams
    
def print_top_categories(model, img_tensor, top_k=5):
    logits = model(img_tensor.unsqueeze(0)).logits
    indices = logits.cpu()[0, :].detach().numpy().argsort()[-top_k :][::-1]
    for i in indices:
        print(f"Predicted class {i}: {model.config.id2label[i]}")

# Generate targets_for_gradcam based on model's predictions
def get_top_k_targets(model, input_tensor, k=5):
    logits = model(input_tensor.unsqueeze(0)).logits
    top_k_indices = logits[0].argsort(descending=True)[:k].cpu().numpy()
    return [ClassifierOutputTarget(index) for index in top_k_indices]

import os
from tqdm import tqdm
from collections import defaultdict
import gc
###################
from transformers import ViTFeatureExtractor, ViTForImageClassification
from functools import partial
import torch
from PIL import Image
import datetime

###################

def reshape_transform_vit_huggingface(x):
    activations = x[:, 1:, :]
    activations = activations.view(activations.shape[0],
                                   12, 12, activations.shape[2])
    activations = activations.transpose(2, 3).transpose(1, 2)
    return activations



model_name = "google/vit-large-patch32-384"

# Initialize tracking variables
start_time = datetime.datetime.now()
emissions_reports = []




def ensure_rgb(img):
    if img.mode != 'RGB':
        return img.convert('RGB')
    return img

def is_valid_image_file(filepath):
    """Check if the file is a valid image file."""
    try:
        with Image.open(filepath) as img:
            img.verify()  # verify that it is a valid image
        return True
    except:
        return False

BATCH_SIZE = 100
num_batches = len(dataset) // BATCH_SIZE + (1 if len(dataset) % BATCH_SIZE != 0 else 0)

save_dir = f"{current_dir}/results/{detail_dir}/{model_name}/{cam_algorithm_name}"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, len(dataset))

    model = ViTForImageClassification.from_pretrained('google/vit-large-patch32-384').to(device)

    target_layer_dff = model.vit.layernorm
    target_layer_gradcam = model.vit.encoder.layer[-2].output




    for idx in range(start_idx, end_idx):
        img, label, filename = dataset[idx]
        try:
            torch.cuda.empty_cache()
            img = ensure_rgb(img)
            image_resized = img.resize((384, 384))
            tensor_resized = transforms.ToTensor()(image_resized).to(device)
            # resize_transform = transforms.Resize((480, 640))
            # img = resize_transform(img)
            # img_tensor = transform(img).to(device)
            #print(img_tensor.shape)


            index_description = label_to_index_description.get(label)
            if index_description is None:
                print(f"Warning: Label '{label}' not found in the JSON file!")
                continue

            index_str, description = index_description
            index = int(index_str)
            dynamic_targets_for_gradcam = [ClassifierOutputTarget(index)]

            
            gradcam_result, grayscale_cams = run_grad_cam_on_image(
                model=model,
                target_layer=target_layer_gradcam,
                targets_for_gradcam=dynamic_targets_for_gradcam,
                input_tensor=tensor_resized,
                input_image=image_resized,
                reshape_transform=reshape_transform_vit_huggingface
            )

            logits = model(tensor_resized.unsqueeze(0)).logits
            top_indices = logits[0].argsort(descending=True)[:5].cpu().numpy()
            predictions = {index: {"score": logits[0][index].item(), "label": model.config.id2label[index]} for index in top_indices}
            
            img_dir = os.path.join(save_dir, filename.rsplit('.', 1)[0])
            if not os.path.exists(img_dir):
                os.makedirs(img_dir)

            img_name = os.path.join(img_dir, "original.jpg")
            gradcam_name = os.path.join(img_dir, "gradcam.jpg")
            grayscale_name = os.path.join(img_dir, "grayscale.jpg")
            grayscale_npy_name = os.path.join(img_dir, "grayscale.npy")
            scores_name = os.path.join(img_dir, "scores.npy")
            info_name = os.path.join(img_dir, "info.txt")

            img.save(img_name)
            Image.fromarray(gradcam_result).save(gradcam_name)
            Image.fromarray((grayscale_cams[0] * 255).astype(np.uint8)).save(grayscale_name)
            np.save(grayscale_npy_name, grayscale_cams[0])

            scores = [data["score"] for _, data in predictions.items()]
            np.save(scores_name, scores)

            with open(info_name, 'w') as f:
                for index, data in predictions.items():
                    label = data["label"]
                    score = data["score"]
                    f.write(f"Class {index} ({label}): {score:.2f}\n")

        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print(f"CUDA OutOfMemoryError encountered for file: {filename}")
            else:
                raise e

    # del model
    torch.cuda.empty_cache()
    gc.collect()


end_time = datetime.datetime.now()


print(f"Total time: {end_time - start_time}")




100%|██████████| 62/62 [11:01<00:00, 10.66s/it]

Total time: 0:11:01.068262


In [5]:
# #    GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
#     AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
#     LayerCAM, FullGrad, GradCAMElementWise

# #    GradCAM, HiResCAM, GradCAMPlusPlus, XGradCAM, LayerCAM
CAM_ALGORITHM = XGradCAM
cam_algorithm_name = CAM_ALGORITHM.__name__

# @track_emissions()
def run_grad_cam_on_image(model: torch.nn.Module,
                          target_layer: torch.nn.Module,
                          targets_for_gradcam: List[Callable],
                          input_tensor: torch.nn.Module,
                          input_image: Image,
                          reshape_transform: Optional[Callable] = None,
                          method: Callable = CAM_ALGORITHM):
    with method(model=HuggingfaceToTensorModelWrapper(model),
                target_layers=[target_layer],
                reshape_transform=reshape_transform) as cam:

        # Replicate the tensor for each of the categories we want to create Grad-CAM for:
        repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

        batch_results = cam(input_tensor=repeated_tensor,
                            targets=targets_for_gradcam)
        results = []
        grayscale_cams = []
        for grayscale_cam in batch_results:
            visualization = show_cam_on_image(np.float32(input_image) / 255,
                                              grayscale_cam,
                                              use_rgb=True)
            # Make it weight less in the notebook:
            visualization = cv2.resize(visualization,
                                       (visualization.shape[1] // 2, visualization.shape[0] // 2))
            results.append(visualization)
            grayscale_cams.append(grayscale_cam)
        return np.hstack(results), grayscale_cams
    
def print_top_categories(model, img_tensor, top_k=5):
    logits = model(img_tensor.unsqueeze(0)).logits
    indices = logits.cpu()[0, :].detach().numpy().argsort()[-top_k :][::-1]
    for i in indices:
        print(f"Predicted class {i}: {model.config.id2label[i]}")

# Generate targets_for_gradcam based on model's predictions
def get_top_k_targets(model, input_tensor, k=5):
    logits = model(input_tensor.unsqueeze(0)).logits
    top_k_indices = logits[0].argsort(descending=True)[:k].cpu().numpy()
    return [ClassifierOutputTarget(index) for index in top_k_indices]

import os
from tqdm import tqdm
from collections import defaultdict
import gc
###################
from transformers import ViTFeatureExtractor, ViTForImageClassification
from functools import partial
import torch
from PIL import Image
import datetime

###################

def reshape_transform_vit_huggingface(x):
    activations = x[:, 1:, :]
    activations = activations.view(activations.shape[0],
                                   12, 12, activations.shape[2])
    activations = activations.transpose(2, 3).transpose(1, 2)
    return activations



model_name = "google/vit-large-patch32-384"

# Initialize tracking variables
start_time = datetime.datetime.now()
emissions_reports = []




def ensure_rgb(img):
    if img.mode != 'RGB':
        return img.convert('RGB')
    return img

def is_valid_image_file(filepath):
    """Check if the file is a valid image file."""
    try:
        with Image.open(filepath) as img:
            img.verify()  # verify that it is a valid image
        return True
    except:
        return False

BATCH_SIZE = 100
num_batches = len(dataset) // BATCH_SIZE + (1 if len(dataset) % BATCH_SIZE != 0 else 0)

save_dir = f"{current_dir}/results/{detail_dir}/{model_name}/{cam_algorithm_name}"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, len(dataset))

    model = ViTForImageClassification.from_pretrained('google/vit-large-patch32-384').to(device)

    target_layer_dff = model.vit.layernorm
    target_layer_gradcam = model.vit.encoder.layer[-2].output




    for idx in range(start_idx, end_idx):
        img, label, filename = dataset[idx]
        try:
            torch.cuda.empty_cache()
            img = ensure_rgb(img)
            image_resized = img.resize((384, 384))
            tensor_resized = transforms.ToTensor()(image_resized).to(device)
            # resize_transform = transforms.Resize((480, 640))
            # img = resize_transform(img)
            # img_tensor = transform(img).to(device)
            #print(img_tensor.shape)


            index_description = label_to_index_description.get(label)
            if index_description is None:
                print(f"Warning: Label '{label}' not found in the JSON file!")
                continue

            index_str, description = index_description
            index = int(index_str)
            dynamic_targets_for_gradcam = [ClassifierOutputTarget(index)]

            
            gradcam_result, grayscale_cams = run_grad_cam_on_image(
                model=model,
                target_layer=target_layer_gradcam,
                targets_for_gradcam=dynamic_targets_for_gradcam,
                input_tensor=tensor_resized,
                input_image=image_resized,
                reshape_transform=reshape_transform_vit_huggingface
            )

            logits = model(tensor_resized.unsqueeze(0)).logits
            top_indices = logits[0].argsort(descending=True)[:5].cpu().numpy()
            predictions = {index: {"score": logits[0][index].item(), "label": model.config.id2label[index]} for index in top_indices}
            
            img_dir = os.path.join(save_dir, filename.rsplit('.', 1)[0])
            if not os.path.exists(img_dir):
                os.makedirs(img_dir)

            img_name = os.path.join(img_dir, "original.jpg")
            gradcam_name = os.path.join(img_dir, "gradcam.jpg")
            grayscale_name = os.path.join(img_dir, "grayscale.jpg")
            grayscale_npy_name = os.path.join(img_dir, "grayscale.npy")
            scores_name = os.path.join(img_dir, "scores.npy")
            info_name = os.path.join(img_dir, "info.txt")

            img.save(img_name)
            Image.fromarray(gradcam_result).save(gradcam_name)
            Image.fromarray((grayscale_cams[0] * 255).astype(np.uint8)).save(grayscale_name)
            np.save(grayscale_npy_name, grayscale_cams[0])

            scores = [data["score"] for _, data in predictions.items()]
            np.save(scores_name, scores)

            with open(info_name, 'w') as f:
                for index, data in predictions.items():
                    label = data["label"]
                    score = data["score"]
                    f.write(f"Class {index} ({label}): {score:.2f}\n")

        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print(f"CUDA OutOfMemoryError encountered for file: {filename}")
            else:
                raise e

    # del model
    torch.cuda.empty_cache()
    gc.collect()


end_time = datetime.datetime.now()


print(f"Total time: {end_time - start_time}")




100%|██████████| 62/62 [10:54<00:00, 10.55s/it]

Total time: 0:10:54.246505


In [6]:
# #    GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
#     AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
#     LayerCAM, FullGrad, GradCAMElementWise

# #    GradCAM, HiResCAM, GradCAMPlusPlus, XGradCAM, LayerCAM
CAM_ALGORITHM = LayerCAM
cam_algorithm_name = CAM_ALGORITHM.__name__

# @track_emissions()
def run_grad_cam_on_image(model: torch.nn.Module,
                          target_layer: torch.nn.Module,
                          targets_for_gradcam: List[Callable],
                          input_tensor: torch.nn.Module,
                          input_image: Image,
                          reshape_transform: Optional[Callable] = None,
                          method: Callable = CAM_ALGORITHM):
    with method(model=HuggingfaceToTensorModelWrapper(model),
                target_layers=[target_layer],
                reshape_transform=reshape_transform) as cam:

        # Replicate the tensor for each of the categories we want to create Grad-CAM for:
        repeated_tensor = input_tensor[None, :].repeat(len(targets_for_gradcam), 1, 1, 1)

        batch_results = cam(input_tensor=repeated_tensor,
                            targets=targets_for_gradcam)
        results = []
        grayscale_cams = []
        for grayscale_cam in batch_results:
            visualization = show_cam_on_image(np.float32(input_image) / 255,
                                              grayscale_cam,
                                              use_rgb=True)
            # Make it weight less in the notebook:
            visualization = cv2.resize(visualization,
                                       (visualization.shape[1] // 2, visualization.shape[0] // 2))
            results.append(visualization)
            grayscale_cams.append(grayscale_cam)
        return np.hstack(results), grayscale_cams
    
def print_top_categories(model, img_tensor, top_k=5):
    logits = model(img_tensor.unsqueeze(0)).logits
    indices = logits.cpu()[0, :].detach().numpy().argsort()[-top_k :][::-1]
    for i in indices:
        print(f"Predicted class {i}: {model.config.id2label[i]}")

# Generate targets_for_gradcam based on model's predictions
def get_top_k_targets(model, input_tensor, k=5):
    logits = model(input_tensor.unsqueeze(0)).logits
    top_k_indices = logits[0].argsort(descending=True)[:k].cpu().numpy()
    return [ClassifierOutputTarget(index) for index in top_k_indices]

import os
from tqdm import tqdm
from collections import defaultdict
import gc
###################
from transformers import ViTFeatureExtractor, ViTForImageClassification
from functools import partial
import torch
from PIL import Image
import datetime

###################

def reshape_transform_vit_huggingface(x):
    activations = x[:, 1:, :]
    activations = activations.view(activations.shape[0],
                                   12, 12, activations.shape[2])
    activations = activations.transpose(2, 3).transpose(1, 2)
    return activations



model_name = "google/vit-large-patch32-384"

# Initialize tracking variables
start_time = datetime.datetime.now()
emissions_reports = []




def ensure_rgb(img):
    if img.mode != 'RGB':
        return img.convert('RGB')
    return img

def is_valid_image_file(filepath):
    """Check if the file is a valid image file."""
    try:
        with Image.open(filepath) as img:
            img.verify()  # verify that it is a valid image
        return True
    except:
        return False

BATCH_SIZE = 100
num_batches = len(dataset) // BATCH_SIZE + (1 if len(dataset) % BATCH_SIZE != 0 else 0)

save_dir = f"{current_dir}/results/{detail_dir}/{model_name}/{cam_algorithm_name}"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * BATCH_SIZE
    end_idx = min((batch_num + 1) * BATCH_SIZE, len(dataset))

    model = ViTForImageClassification.from_pretrained('google/vit-large-patch32-384').to(device)

    target_layer_dff = model.vit.layernorm
    target_layer_gradcam = model.vit.encoder.layer[-2].output




    for idx in range(start_idx, end_idx):
        img, label, filename = dataset[idx]
        try:
            torch.cuda.empty_cache()
            img = ensure_rgb(img)
            image_resized = img.resize((384, 384))
            tensor_resized = transforms.ToTensor()(image_resized).to(device)
            # resize_transform = transforms.Resize((480, 640))
            # img = resize_transform(img)
            # img_tensor = transform(img).to(device)
            #print(img_tensor.shape)


            index_description = label_to_index_description.get(label)
            if index_description is None:
                print(f"Warning: Label '{label}' not found in the JSON file!")
                continue

            index_str, description = index_description
            index = int(index_str)
            dynamic_targets_for_gradcam = [ClassifierOutputTarget(index)]

            
            gradcam_result, grayscale_cams = run_grad_cam_on_image(
                model=model,
                target_layer=target_layer_gradcam,
                targets_for_gradcam=dynamic_targets_for_gradcam,
                input_tensor=tensor_resized,
                input_image=image_resized,
                reshape_transform=reshape_transform_vit_huggingface
            )

            logits = model(tensor_resized.unsqueeze(0)).logits
            top_indices = logits[0].argsort(descending=True)[:5].cpu().numpy()
            predictions = {index: {"score": logits[0][index].item(), "label": model.config.id2label[index]} for index in top_indices}
            
            img_dir = os.path.join(save_dir, filename.rsplit('.', 1)[0])
            if not os.path.exists(img_dir):
                os.makedirs(img_dir)

            img_name = os.path.join(img_dir, "original.jpg")
            gradcam_name = os.path.join(img_dir, "gradcam.jpg")
            grayscale_name = os.path.join(img_dir, "grayscale.jpg")
            grayscale_npy_name = os.path.join(img_dir, "grayscale.npy")
            scores_name = os.path.join(img_dir, "scores.npy")
            info_name = os.path.join(img_dir, "info.txt")

            img.save(img_name)
            Image.fromarray(gradcam_result).save(gradcam_name)
            Image.fromarray((grayscale_cams[0] * 255).astype(np.uint8)).save(grayscale_name)
            np.save(grayscale_npy_name, grayscale_cams[0])

            scores = [data["score"] for _, data in predictions.items()]
            np.save(scores_name, scores)

            with open(info_name, 'w') as f:
                for index, data in predictions.items():
                    label = data["label"]
                    score = data["score"]
                    f.write(f"Class {index} ({label}): {score:.2f}\n")

        except RuntimeError as e:
            if "CUDA out of memory" in str(e):
                print(f"CUDA OutOfMemoryError encountered for file: {filename}")
            else:
                raise e

    # del model
    torch.cuda.empty_cache()
    gc.collect()


end_time = datetime.datetime.now()


print(f"Total time: {end_time - start_time}")




100%|██████████| 62/62 [11:11<00:00, 10.83s/it]

Total time: 0:11:11.669215
